In [ ]:
import pandas as pd
df = pd.read_csv('research candidate sets.csv')
print(df.shape)
print(df.dtypes)

In [ ]:
for index, row in df.iterrows():
    doc_id = row.doc_id
    
    import ast
    authorList = ast.literal_eval(row.authorList)
    
    val = 4
    
    test = "trainableoff"
    
    level = "word"
    
    iterations = 30
    
    printstate = (("doc_id = %s, candidate = %s, ") % (str(doc_id), str(val)))
    printstate += (("dimensions = %s, samples = %s, ") % (str(200), str(300)))
    printstate += (("\niterations = %s, dropout = %s, test = %s") % (str(180), str(0.2), str(test)))
    
    print("Current test: %s" % (str(printstate)))
    
    import UpdateDB as db
    
    case = db.checkLSTM(doc_id = doc_id, candidate = val, dimensions = 200, samples = 300, 
                        iterations = iterations, dropout = 0.2, test = test)
    
    if case == False:
        
        print("Running: %12s" % (str(printstate)))
        
        import StyloNeural as Stylo
        (labels_index, predYList, predY, history) = Stylo.getResults(authorList = authorList[:(val + 1)], 
                                                                     doc_id = doc_id, 
                                                                     level = level, 
                                                                     glove = '../../glove/',
                                                                     nb_epoch = iterations,
                                                                     samples = 300, 
                                                                     dimensions = 200,
                                                                     dropout = 0.2, 
                                                                     batch_size = 30
                                                                    )
        
        import UpdateDB as db
        db.updateresultLSTM(doc_id = doc_id, candidate = val, dimensions = 200, samples = 300, 
                            iterations = iterations, dropout = 0.2, accuracy = predY[0], test = test)
        
        from IPython.display import clear_output
        clear_output()
    
    else:
        print("Skipped: %12s" % (str(printstate)))
        

In [ ]:
# import matplotlib.pyplot as plt
# # summarize history for accuracy
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [ ]:
# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [ ]:
%tb